In [ ]:
# CNN Classification

In [1]:
# IMPORTS
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import layers, models
from keras import backend as K
from keras import regularizers
from keras.layers.noise import AlphaDropout
from collections import namedtuple
import math
import matplotlib.pyplot as plt
import pickle
import datetime
import os
import seaborn as sns
Point = namedtuple('Point', ('x', 'y'))
Circle = namedtuple('Circle', ('r'))
Square = namedtuple('Square', ('side'))
Rectangle = namedtuple('Rectangle', ('length', 'width'))
PointWithDistance = namedtuple('PointWithDistance', ('p', 'dist'))
float_memory_used = 'float16'

Using TensorFlow backend.


In [2]:
tf.test.is_gpu_available()

True

In [ ]:
# INIT
number_samples = [5] + list(range(10, 101, 10)) + [120, 150, 200, 250, 300, 400, 500, 700] + list(range(1000, 1001, 1000))
number_samples = [120, 150, 200, 250, 300, 400, 500, 700] + list(range(1000, 1001, 1000))
number_samples = [500]

cnn_type = "classification"  # {"classification", "regression"}
validation_size, noise_floor = 0.33, -90.0
su_power = 0 # this is not actually su power just a number to show there is an SU in its image
max_x, max_y, number_image_channels, su_szie = 200, 200, 2, None
pu_shape, su_shape = 'circle', 'circle' # shape = {'circle', 'square', 'point'}
style = "raw_power_min_max_norm"  # {"raw_power_zscore_norm", "image_intensity", "raw_power_min_max_norm"}
pus_num, intensity_degradation, slope = 15, 'log', 6
if su_shape == 'circle':
    su_param = Circle(su_szie)
elif su_shape == 'square':
    su_param = Square(su_szie)
else:
    su_param = None
# num_pus = (data_reg.shape[1] - 3)//3
dtime = datetime.datetime.now().strftime('_%Y%m_%d%H_%M')
color = "color" if number_image_channels > 1 else "grey"
image_dir = 'ML/data/'+ cnn_type + '/pictures_' + str(max_x) + '_' + str(max_y) + '/pu_' + pu_shape + '_su_' + su_shape + '_'  + "/" + style + "/" + color +'/' + (
    "" if pu_shape == 'point' and su_shape == 'point' else (intensity_degradation + '_' + str(slope)))

In [ ]:
# LOAD DATA
dataframe = pd.read_csv('ML/data/static_pus_using_pus50000_15PUs_201912_2315_33_200.txt', delimiter=',', header=None)
dataframe_max = pd.read_csv('ML/data/static_pus_max_power50000_15PUs_201912_2315_33_200.txt', delimiter=',', header=None)

dataframe.reset_index(drop=True, inplace=True)
dataframe_max.reset_index(drop=True, inplace=True)

dataframe_tot = pd.concat([dataframe, dataframe_max.iloc[:, dataframe_max.columns.values[-1]]], axis=1,
                        ignore_index=True)
idx = dataframe_tot[dataframe_tot[dataframe_tot.columns[-1]] == -float('inf')].index
dataframe_tot.drop(idx, inplace=True)

# data_reg = np.concatenate((dataframe_tot.values[:, 0:dataframe_tot.shape[1]-3], 
#                            dataframe_tot.values[:, dataframe_tot.shape[1]-1:dataframe_tot.shape[1]]), axis=1)
data_class = dataframe_tot.values[:, 0:dataframe_tot.shape[1]-1]
y_class_power = dataframe_tot.values[:, -1]
del dataframe, dataframe_max

In [ ]:
max_su_power = -float('inf')
for i in range(data_reg.shape[0]):
    max_su_power = max(su_max_powe, data_reg[i][-1])

In [ ]:
def euclidian_distance(p1: Point, p2: Point):
    return ((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2) ** 0.5

def calculate_mu_sigma(data, num_pus):
    sum_non_noise = 0
    for pu_n in range(num_pus): # calculate mu
        sum_non_noise += data[pu_n*3+2]
    mu = ((max_x * max_y - num_pus) * noise_floor + sum_non_noise)/(max_x * max_y)
    sum_square = 0
    for pu_n in range(num_pus): # calculate sigma
        sum_square += (data[pu_n*3+2]-mu)**2
    sum_square += (max_x * max_y - num_pus) * (noise_floor - mu)**2
    sigma = math.sqrt(sum_square/(max_x * max_y))
    return mu, sigma

def get_pu_param(pu_shape: str, intensity_degradation: str, pu_p: float, noise_floor: float, slope: float):
    pu_param = None
    if pu_shape == 'circle':
        if intensity_degradation == "linear":
            pu_param = Circle(int((pu_p - noise_floor) / slope)) # linear
        elif intensity_degradation == "log":
            pu_param = Circle(int(10 ** ((pu_p - noise_floor) / (10 *slope)))) # log_based
    elif pu_shape == 'square':
        if intensity_degradation == "linear":
            pu_param = Square(int(2 ** 0.5 * (pu_p - noise_floor) / slope)) # linear
        elif intensity_degradation == "log":
            pu_param = Square(int(2 ** 0.5 * 10 ** ((pu_p - noise_floor) / (10 *slope)))) # log_based
    elif pu_shape == 'point':
        pu_param = None
    else:
        raise ValueError("Unsupported PU shape(create_image)! ", pu_shape)
    return pu_param

def create_image(data, pus_num, slope, style="raw_power_z_score", noise_floor=-90, pu_shape= 'circle', pu_param=None, 
                 su_shape='circle', su_param=None, intensity_degradation="log", max_pu_power: float=0.0, max_su_power=0.0):  
    # style = {"raw_power_zscore_norm", "image_intensity", "raw_power_min_max_norm"}
    # intensity_degradation= {"log", "linear"}
    # if param is None, it's automatically calculated. Highest brightness(or power value) (255 or 1.) would
    # assigned to the center(PU location) and radius(side) would be calculated based on its power, slope, and noise floor.
    # If it is given, intensity(power) of pixel beside center would be calculated in the same fashin with an exception that 
    # intensity below zero(noise_floor) would be replaced by zero(noise_floor)
    if style == "raw_power_min_max_norm":
        # In this way, PUs' location are replaced with their power(dBm) and the power would fade with 
        # slope till gets noise_floor(in circle shape)
        
        # creating pu matrix
        image = np.zeros((1,number_image_channels,max_x, max_y), dtype=float_memory_used)
        for pu_i in range(pus_num):
            pu_x, pu_y, pu_p = max(0, min(max_x-1, int(data[pu_i*3]))), max(0, min(max_x-1, int(data[pu_i*3+1]))), data[
                pu_i*3+2]
            if not pu_param:
                pu_param_p = get_pu_param(pu_shape, intensity_degradation, pu_p, noise_floor, slope)
            else:
                pu_param_p = pu_param
            points = points_inside_shape(center=Point(pu_x, pu_y), shape=pu_shape, param=pu_param_p)
            for point in points:
                if 0 <= point.p.x < max_x and 0 <= point.p.y < max_y: # TODO should pass image size
                    if intensity_degradation == "linear":
                        image[0][0][point.p.x][point.p.y] += (pu_p - slope * point.dist - noise_floor)/(
                            max_pu_power - noise_floor)
                    elif intensity_degradation == "log":
                        if point.dist < 1:
                            image[0][0][point.p.x][point.p.y] += (pu_p - noise_floor) / (max_pu_power - noise_floor)
                        else:
                            image[0][0][point.p.x][point.p.y] += (pu_p - slope * 10*math.log10(point.dist) - noise_floor)/(
                                max_pu_power - noise_floor)
        # creating su matrix
        su_num = (len(data) - pus_num * 3 - 1) // 3
        if not (len(data) - pus_num * 3) % 3:
            raise ValueError("Data provided is not correct; can't get SUs' information(create_image)")
#         if su_param is None:
#             # if su_param is unavailable, a circle(square) with radius(side) 1 is created
#             if su_shape == 'circle':
#                 su_param = Circle(1)
#             elif su_shape == 'square':
#                 su_param = Square(1)
#             elif su_shape == 'point':
#                 su_param = None
#             else:
#                 raise ValueError("Unsupported SU shape(create_image)! ", su_shape)
#         su_intensity = 1.
        su_channel = 0 if number_image_channels == 1 else 1
        for su_i in range(su_num):
            su_x, su_y, su_p = max(0, min(max_x-1, int(data[pus_num*3+su_i*2]))), max(
                0, min(max_x-1, int(data[pus_num*3+su_i*2+1]))), data[pus_num*3+su_i*2+2]
            if not su_param:
                su_param_s = get_pu_param(su_shape, intensity_degradation, su_p, noise_floor, slope)
            else:
                su_param_s = su_param
            points = points_inside_shape(center=Point(su_x, su_y), param=su_param_s, shape=su_shape)
            for point in points:
                if 0 <= point.p.x < max_x and 0 <= point.p.y < max_y: # TODO should pass image size
                    if intensity_degradation == "linear":
                        image[0][su_channel][point.p.x][point.p.y] += (su_p - slope * point.dist - noise_floor)/(
                            max_su_power - noise_floor)
                    elif intensity_degradation == "log":
                        if point.dist < 1:
                            image[0][su_channel][point.p.x][point.p.y] += (su_p - noise_floor) / (max_su_power - noise_floor)
                        else:
                            image[0][su_channel][point.p.x][point.p.y] += (su_p - slope * 10*math.log10(point.dist) - noise_floor)/(
                                max_su_power - noise_floor)
        return image
        
#         pu_image = [[(noise_floor - mu)/sigma] * max_y for _ in range(max_x)]
    elif style == "image_intensity":
        # creating PU image
        image = np.zeros((1,number_image_channels,max_x, max_y), dtype=float_memory_used)
        for pu_i in range(pus_num):
            pu_x, pu_y, pu_p = max(0, min(max_x-1, int(data[pu_i*3]))), max(0, min(max_x-1, int(data[pu_i*3+1]))), data[pu_i*3+2]
            if pu_param is None:
                pu_param = get_pu_param(pu_shape, intensity_degradation, pu_p, noise_floor, slope)
            points = points_inside_shape(center=Point(pu_x, pu_y), shape=pu_shape, param=pu_param)
            for point in points:
                if 0 <= point.p.x < max_x and 0 <= point.p.y < max_y: # TODO should pass image size
                    if intensity_degradation == "linear":
                        image[0][0][point.p.x][point.p.y] += max((pu_p - slope * point.dist + abs(noise_floor))
                                                              /(pu_p + abs(noise_floor)), 0)
                    elif intensity_degradation == "log":
                        if point.dist < 1:
                            image[0][0][point.p.x][point.p.y] = 1
                        else:
                            image[0][0][point.p.x][point.p.y] += max((pu_p - slope * 10*math.log10(point.dist) + abs(noise_floor))
                                                                 /(pu_p + abs(noise_floor)), 0)
                    image[0][0][point.p.x][point.p.y] = min(image[0][0][point.p.x][point.p.y], 1.0)
                        
        # creating SU image
        su_num = (len(data) - pus_num * 3) // 3
        if not (len(data) - pus_num * 3) % 3:
            raise ValueError("Data provided is not correct; can't get SUs' information(create_image)")
#         su_image = np.zeros((max_x, max_y), dtype=float_memory_used)
#         if su_param is None:
#             # if su_param is unavailable, a circle(square) with radius(side) 1 is created
#             if su_shape == 'circle':
#                 su_param = Circle(1)
#             elif su_shape == 'square':
#                 su_param = Square(1)
#             elif su_shape == 'point':
#                 su_param = None
#             else:
#                 raise ValueError("Unsupported SU shape(create_image)! ", su_shape)
        su_channel = 0 if number_image_channels == 1 else 1
#         su_intensity = 1.
        for su_i in range(su_num):
            su_x, su_y, su_p = max(0, min(max_x-1, int(data[pus_num*3+su_i*2]))
                                  ), max(0, min(max_x-1, int(data[pus_num*3+su_i*2+1]))), su_intensity
            points = points_inside_shape(center=Point(su_x, su_y), param=su_param, shape=su_shape)
            for point in points:
                if 0 <= point.p.x < max_x and 0 <= point.p.y < max_y: # TODO should pass image size
                    if number_image_channels > 1:
                        image[0][1][point.p.x][point.p.y] = su_intensity
                    elif number_image_channels == 1:
                        image[0][0][point.p.x][point.p.y] = su_intensity
#         return np.array([pu_image, su_image, [[0.] * max_y for _ in range(max_x)]], dtype='float32') # return like this to be able to display as an RGB image with pyplot.imshow(imsave)
#         return np.append(pu_image, su_image, axis=0)
        return image
        
            
    else:
        raise ValueError("Unsupported style(create_image)! ", style)
        
def points_inside_shape(center: Point, shape: str, param)-> list:
    # This function returns points+distance around center with defined shape
    if shape == 'circle':
        # First creates points inside a square(around orgigin) with 2*r side and then remove those with distance > r.
        # Shift all remaining around center. O(4r^2)
        r, origin = param.r, Point(0, 0)
        square_points = set([Point(x, y) for x in range(-r, r+1) for y in range(-r, r+1)])
        points = []
        while square_points:
            p = square_points.pop()
            dist = euclidian_distance(p, origin)
            if dist <= r:
                points.append(PointWithDistance(Point(p.x + center.x, p.y + center.y), dist))
                if p.x != 0:
                    points.append(PointWithDistance(Point(-p.x + center.x, p.y + center.y), dist))
                    square_points.remove(Point(-p.x, p.y))
                if p.y != 0:
                    points.append(PointWithDistance(Point(p.x + center.x, -p.y + center.y), dist))
                    square_points.remove(Point(p.x, -p.y))
                if p.x != 0 and p.y != 0:
                    points.append(PointWithDistance(Point(-p.x + center.x, -p.y + center.y), dist))
                    square_points.remove(Point(-p.x, -p.y))
        return points
    elif shape == 'square':
        half_side = param.side // 2
        return [PointWithDistance(Point(x, y), euclidian_distance(Point(x, y), center)) for x in range(-half_side + center.x,
                                                                                               half_side + center.x+1) 
                         for y in range(-half_side + center.y, half_side + center.y + 1)]
    elif shape == 'point':
        return [PointWithDistance(center, 0)]
    else:
        raise ValueError("Unsupported shape(points_inside_shape)! ", shape)
    
# TODO: Consider using min_max normalization becasue difference between values using
# z-score is huge since most of the pixels have the same value, noise floor

In [ ]:
# Saving images once to save time
# run this cell just for creating images
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
# for image_num in range(115, data_reg.shape[0]):
for image_num in range(0, 1330):
    image = create_image(data=data_class[image_num], slope=slope, style=style, noise_floor=noise_floor,
                         pu_shape=pu_shape, su_shape=su_shape, su_param=None, pus_num=pus_num, 
                         intensity_degradation=intensity_degradation, max_su_power=50.0)
    if style == "image_intensity":
        if number_image_channels != 3:
            image = np.append(np.array(image[0]), np.zeros((3-number_image_channels,max_x, max_y), 
                                                           dtype=float_memory_used), axis=0)
        image_save = np.swapaxes(image, 0, 2)
        plt.imsave(image_dir + '/image' + str(image_num)+'.png', image_save)
    elif style == "raw_power_min_max_norm" or style == "raw_power_zscore_norm":
        np.save(image_dir + '/image' + str(image_num), image)
    if image_num + 1 % 100 == 0:
        print(image_num)

In [ ]:
fig, ax = plt.subplots(figsize=(70,70))         # Sample figsize in inches
sns.heatmap(image[0][0], ax=ax)

In [ ]:
# TODO, CNN Model here
def cnn_model(num_filters, kernel_lam, bias_lam):
#     num_filters, lam = 5, 5
    data_format = 'channels_first'
    convolution_filter, dense_filter = 'selu', 'sigmoid' #softsign, sigmoid; relu, linear
    filter_shape, pool_size = (3, 3), (2,2)
    cnn = models.Sequential()
    cnn.add(layers.Conv2D(num_filters, filter_shape, padding='same', activation=convolution_filter, 
                          input_shape=(number_image_channels, max_x, max_y), data_format=data_format, 
                          kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                          kernel_initializer='lecun_normal'))
    cnn.add(layers.MaxPooling2D(pool_size=pool_size, data_format=data_format))
    
    cnn.add(layers.Conv2D(num_filters, filter_shape,padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer='lecun_normal'))
    cnn.add(layers.MaxPooling2D(pool_size=pool_size, data_format=data_format))
    
    cnn.add(layers.Conv2D(num_filters, filter_shape, padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer='lecun_normal'))
    cnn.add(layers.MaxPooling2D(pool_size, data_format=data_format))
    
    cnn.add(layers.Conv2D(num_filters, filter_shape, padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer='lecun_normal'))
    cnn.add(layers.MaxPooling2D(pool_size, data_format=data_format))
    
    cnn.add(layers.Conv2D(num_filters, filter_shape, padding='same', activation=convolution_filter, data_format=data_format, 
                         kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
                         kernel_initializer='lecun_normal'))
    cnn.add(layers.MaxPooling2D(pool_size, data_format=data_format))
    
#     cnn.add(layers.Conv2D(num_filters, filter_shape, padding='same', activation=convolution_filter, data_format=data_format, 
#                          kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
#                          kernel_initializer='lecun_normal'))
#     cnn.add(layers.MaxPooling2D(pool_size, data_format=data_format))
    
#     cnn.add(layers.Conv2D(num_filters, filter_shape, padding='same', activation=convolution_filter, data_format=data_format, 
#                          kernel_regularizer=regularizers.l2(kernel_lam), bias_regularizer=regularizers.l2(bias_lam),
#                           kernel_initializer='lecun_normal'))
#     cnn.add(layers.MaxPooling2D(pool_size, data_format=data_format))
#     cnn.add(AlphaDropout(0.25))
    cnn.add(layers.Flatten())
#     cnn.add(layers.Dense(20, activation=convolution_filter, kernel_regularizer=regularizers.l2(lam), bias_regularizer=regularizers.l2(lam)))
    cnn.add(layers.Dense(20, activation=convolution_filter, kernel_regularizer=regularizers.l2(kernel_lam),
                         bias_regularizer=regularizers.l2(bias_lam), kernel_initializer='lecun_normal'))
#     cnn.add(AlphaDropout(0.25))
    cnn.add(layers.Dense(1, activation=dense_filter, kernel_regularizer=regularizers.l2(kernel_lam),
                         bias_regularizer=regularizers.l2(bias_lam), kernel_initializer='lecun_normal'))
    return cnn

In [ ]:
cnn = cnn_model(7, 0, 0)

cnn.summary()

In [ ]:
# CNN
prev_sample = 0
lambda_vec = [0,0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3]
max_train_samples = math.ceil(number_samples[-1] * (1 + validation_size))
x_train = np.empty((max_train_samples, number_image_channels, max_x, max_y), dtype=float_memory_used)
y_train = np.empty((max_train_samples), dtype=float_memory_used)
average_diff_power, fp_mean_power = [], []
for number_sample in number_samples:
    sample = math.ceil(number_sample * (1 + validation_size))
    for image_num in range(prev_sample, sample):
        prev_sample = sample
        if style == "image_intensity":
            image = plt.imread(image_dir + '/image' + str(image_num)+'.png')
            image = np.swapaxes(image, 0, 2)
            x_train[image_num] = np.array(image[:number_image_channels], dtype=float_memory_used).reshape(1, number_image_channels, max_x, max_y)
            del image
        elif  style == "raw_power_min_max_norm" or style == "raw_power_zscore_norm":
            x_train[image_num] = np.load(image_dir + '/image' + str(image_num)+'.npy')
        y_train[image_num] = np.asarray(data_class[image_num][-1], dtype=float_memory_used)
        if image_num + 1 % 100 == 0:
            print(image_num)
    best_acc = -float('inf')
    best_model, best_lam = None, None
    for lamb in lambda_vec:
#         print("Lambda:", lamb)
        cnn = cnn_model(7, lamb, 0)
        cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        cnn.fit(x_train[:sample], y_train[:sample].astype(int), epochs=1, verbose=0, batch_size=1, 
                validation_split=validation_size/(validation_size + 1))
        if cnn.history.history['val_acc'][-1] > best_acc:
            best_acc = cnn.history.history['val_acc'][-1]
            best_model = cnn
            best_lam = lamb
    print("best lambda", best_lam, "best_acc", best_acc)
    # evaluating test images
    sum_mae, sum_fp_mae = 0, 0
    test_size = 0
#     for test_num in range(max_train_samples, data_reg.shape[0]):
    for test_num in range(1330, 5000):
        test_size += 1
        #Power max calculations
        test_y = y_class_power[test_num]
        test_yp = -90
        max_power = 55  # 10 is added to increase higher bound
        min_power = -90  # 10 is subtracted to decrease lower bound
        h = max_power
        l = min_power
        while h - l > 0.5:
            mid = l + (h - l)/2;
            X_test = data_class[test_num]
            X_test[-2] = mid
            if style == "image_intensity":
                pass
            else:
                test_image = create_image(data=X_test, slope=slope, style=style, noise_floor=noise_floor,
                        pu_shape=pu_shape, su_shape=su_shape, su_param=None, pus_num=pus_num, 
                        intensity_degradation=intensity_degradation, max_su_power=50.0)
            res_tmp = best_model.predict_classes(test_image)[0][0]
            if res_tmp:
                l = mid
                test_yp = max(test_yp, mid)
            else:
                h = mid
        sum_mae += abs(test_yp - test_y)
        if test_yp > test_y:
            sum_fp_mae += abs(test_yp - test_y)
        if test_num % 500 == 0:
            print('test: ', test_num)
        
#         print('Number_samples:', number_sample, ', accuracy:', accuracy[-1], ', f_score:', f_score[-1], 
#               ', fp:', fp,', fn:', fn, ', error:', average_class_diff_power[-1], 'fp_error:', fp_mean_power[-1])
        
    fp_mean_power.append(round(sum_fp_mae/ test_size, 3))
    average_diff_power.append(round(sum_mae / test_size, 3))
    print('number_samples: ', number_sample, ', average_error: ', average_diff_power[-1], ' fp_average_error: ', fp_mean_power[-1])
    
    var_f = open('/'.join(image_dir.split('/')[:-1]) +  '/' + intensity_degradation + '_' + str(slope) + '_' + dtime + ".dat", "wb") # file for saving results
    pickle.dump([average_diff_power, fp_mean_power, number_samples], file=var_f)
    var_f.close()       

In [ ]:
X_test = data_class[3000]
X_test[-2] = 40
test_image = create_image(data=X_test, slope=slope, style=style, noise_floor=noise_floor,
                          pu_shape=pu_shape, su_shape=su_shape, su_param=None, pus_num=pus_num, 
                          intensity_degradation=intensity_degradation, max_su_power=50.0)

In [ ]:
y_class_power[3000]

In [ ]:
cnn.predict_classes(test_image)[0][0]

In [ ]:
fig, ax = plt.subplots(figsize=(70,70))         # Sample figsize in inches
sns.heatmap(x_train[200][0], ax=ax)

In [ ]:
lambda_vec = [0,0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3]
best_acc = -float('inf')
best_model, best_lam = None, None
for lamb in lambda_vec:
    print("Lambda:", lamb)
    cnn = cnn_model(15, lamb, 0)
    cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    cnn.fit(x_train[:sample], y_train[:sample].astype(int), epochs=1, verbose=1, batch_size=1, validation_split=validation_size/
            (validation_size + 1))
    if cnn.history.history['acc'][-1] > best_acc:
        min_error = cnn.history.history['acc'][-1]
        best_model = cnn
        best_lam = lamb
    print("\n")
    

In [ ]:
sum_mae, sum_fp_mae = 0, 0
test_size = 0
for test_num in range(1330, 1700):
        test_size += 1
        #Power max calculations
        test_y = y_class_power[test_num]
        test_yp = -float('inf')
        max_power = 55  # 10 is added to increase higher bound
        min_power = -90  # 10 is subtracted to decrease lower bound
        h = max_power
        l = min_power
        while h - l > 0.5:
            mid = l + (h - l)/2;
            X_test = data_class[image_num]
            X_test[-2] = mid
            if style == "image_intensity":
                pass
            else:
                test_image = create_image(data=X_test, slope=slope, style=style, noise_floor=noise_floor,
                        pu_shape=pu_shape, su_shape=su_shape, su_param=None, pus_num=pus_num, 
                        intensity_degradation=intensity_degradation, max_su_power=50.0)
            res_tmp = cnn.predict_classes(test_image)[0][0]
            if res_tmp:
                l = mid
                test_yp = max(test_yp, mid)
            else:
                h = mid
        sum_mae += abs(test_yp - test_y)
        if test_yp > test_y:
            sum_fp_mae += abs(test_yp - test_y)
        if test_num % 500 == 0:
            print('test: ', test_num)
fp_mean_power.append(round(sum_fp_mae/ test_size, 3))
average_diff_power.append(round(sum_mae / test_size, 3))
print('number_samples: ', number_sample, ', average_error: ', average_diff_power[-1], ' fp_average_error: ', fp_mean_power[-1])
    

In [ ]:
best_model.predict_classes(test_image)[0][0]

In [ ]:
test_y